<div style="font-size: 200%; font-weight: bold; color: maroon;">Modelos de propagación en grafos</div>
<!-- Paulo Villegas, 2020 - v. 2.3 -->

Ejemplo tomado de [KDDComplexNetworkAnalysis](https://colab.research.google.com/github/KDDComplexNetworkAnalysis/CNA_Tutorials/blob/master/NDlib.ipynb).  Usa [NDLib](https://ndlib.readthedocs.io/en/latest/index.html) (Network Diffusion Library), para definir y ejecutar el modelo.

In [ ]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import os

plt.style.use('bmh')
mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['figure.titlesize'] = 'medium'

# This is to bypass the check in ndlib.viz.mpl.DiffusionViz
os.environ['DISPLAY'] = '%'

# Preparación

Necesitamos primero instalar el paquete

In [ ]:
try:
    import ndlib
except ImportError:
    %pip install ndlib==5.1.1

Ahora lo importamos, junto con NetworkX

In [ ]:
import networkx as nx

import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc

# Red de difusión

Definimos el grafo sobre el que ejecutaremos el modelo. Vamos a usar un grafo aleatorio, por simplicidad. Usamos un modelo de "Red de mundo pequeño", para tratar de simular contactos locales pero bajo diámetro

El modelo [Watts-Strogatz](https://en.wikipedia.org/wiki/Watts%E2%80%93Strogatz_model) que usamos tiene 3 parámetros:

     connected_watts_strogatz_graph(n, k, p)

donde 

* `n` : número de nodos del grafo
* `k` : grado medio de un nnodo (cada nodo se conecta inicialmente con sus `k` vecinos más pŕoximos en un anillo)
* `p` : probabilidad de reconexión para cada enlace (formación de contactos lejanos)

In [ ]:
# Network Definition
#g = nx.erdos_renyi_graph(1000, 0.01)

In [ ]:
# Network Definition
g = nx.connected_watts_strogatz_graph(1000, 5, 0.6) 

In [ ]:
%%time

# dibujamos la red, aunque el diagrama queda muy apelotonado
nx.draw(g, node_size=20, edge_color=(0.7,0.7,0.9,0.5), width=0.25)

# Modelo

Vamos a usar un modelo SIR

![SIR](img/sir.png)

In [ ]:
# Model Selection
model = ep.SIRModel(g)

# Model Configuration
config = mc.Configuration()
config.add_model_parameter("fraction_infected", 0.05)

config.add_model_parameter('beta', 0.05)    # infection rate
config.add_model_parameter('gamma', 0.07)    # recovery rate

model.set_initial_status(config)

In [ ]:
model.available_statuses

# Ejecución

In [ ]:
# Simulation
iterations = model.iteration_bunch(50)
trends = model.build_trends(iterations)

# Resultados

In [ ]:
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

viz = DiffusionTrend(model, trends)
viz.plot()

In [ ]:
from ndlib.viz.mpl.DiffusionPrevalence import DiffusionPrevalence

viz = DiffusionPrevalence(model, trends)
viz.plot()